In [20]:
#!pip install -qU crewai

In [2]:
from crewai import Agent, Task, Crew, LLM
from textwrap import dedent

In [3]:
import os
os.environ["GROQ_API_KEY"] = ""  # Replace with your Groq API key


In [4]:
llm = LLM(model="groq/llama-3.1-70b-versatile")

In [5]:
def get_user_requirements():
    print("\n=== Project Requirements ===")
    project_description = input("Enter the project description: ")
    language = input("Enter the programming language (default is Python): ") or "Python"
    specific_requirements = input("Enter any specific requirements (optional): ")

    # Combine all requirements into a detailed task description
    task_description = f"""
    Create a program with the following specifications:

    Project Description: {project_description}
    Programming Language: {language}
    Additional Requirements: {specific_requirements}

    Please provide:
    1. Well-structured code
    2. Proper documentation
    3. Type hints (if applicable)
    4. Error handling
    5. Examples of usage
    """
    return dedent(task_description)

In [6]:

# Define the Developer Agent
developer = Agent(
    role='Senior Software Developer',
    goal='Write clean, efficient, and well-documented code',
    backstory="""You are an experienced developer with strong skills in
    writing maintainable and efficient code. You take feedback well and are
    committed to continuous improvement.""",
    verbose=True,
    llm=llm
)

In [7]:
# Define the Code Reviewer Agent
reviewer = Agent(
    role='Code Reviewer',
    goal='Ensure code quality and suggest improvements',
    backstory="""You are a meticulous code reviewer with years of experience in
    identifying potential issues and suggesting improvements. You focus on code
    quality, performance, and best practices.""",
    verbose=True,
    llm=llm
)

In [8]:
# Get user requirements
user_task_description = get_user_requirements()


=== Project Requirements ===
Enter the project description: create an image with lion and cat
Enter the programming language (default is Python): 
Enter any specific requirements (optional): colorful image


In [9]:

write_code_task = Task(
    description=user_task_description,
    agent=developer,
    expected_output="The expected output is the code generated by the developer agent based on the user's requirements. Don't include any backtick or meta data, just give me the code"
)

In [10]:

# Task 2: Review the code
review_code_task = Task(
    description="""Review the code provided by the developer. Check for:
    1. Code correctness
    2. Performance optimization opportunities
    3. Documentation quality
    4. Type hint usage
    5. Edge cases handling
    Provide specific recommendations for improvement.""",
    agent=reviewer,
    expected_output="Code review feedback and suggestions for improvement.Don't include any backtick or meta data, just give me the code"
)

In [11]:

# Task 3: Implement review feedback
implement_feedback_task = Task(
    description="""Review the feedback provided and implement the suggested
    improvements. Ensure all points are addressed and document any changes
    made.""",
    agent=developer,
    expected_output="Improved code incorporating the review feedback.Don't include any backtick or meta data, just give me the code"
)

In [12]:
# Create the crew
code_crew = Crew(
    agents=[developer, reviewer],
    tasks=[write_code_task, review_code_task, implement_feedback_task],
    verbose=True
)

In [13]:

# Execute the crew's tasks
print("\nStarting the development and review process...")
result = code_crew.kickoff()


Starting the development and review process...
# Agent: Senior Software Developer
## Task: 
Create a program with the following specifications:

Project Description: create an image with lion and cat
Programming Language: Python
Additional Requirements: colorful image

Please provide:
1. Well-structured code
2. Proper documentation
3. Type hints (if applicable)
4. Error handling
5. Examples of usage



# Agent: Senior Software Developer
## Final Answer: 
import numpy as np
from PIL import Image, ImageDraw, ImageFont

def create_image(width: int, height: int) -> Image:
    """
    Creates a new image with the specified width and height.

    Args:
    width (int): The width of the image.
    height (int): The height of the image.

    Returns:
    Image: A new image with the specified width and height.
    """
    return Image.new('RGB', (width, height))

def draw_lion(image: Image, x: int, y: int) -> None:
    """
    Draws a lion on the specified image.

    Args:
    image (Image): 

In [18]:
from PIL import Image, ImageDraw, ImageFont

def create_image(width: int, height: int) -> Image.Image:
    """
    Creates a new RGB image with the specified width and height.

    Args:
    width (int): The width of the image. Must be a non-negative integer.
    height (int): The height of the image. Must be a non-negative integer.

    Returns:
    Image: A new RGB image with the specified width and height.

    Raises:
    ValueError: If the width or height is negative.
    """
    if width < 0 or height < 0:
        raise ValueError("Width and height must be non-negative")
    return Image.new('RGB', (width, height), 'white')


def draw_shape(image: Image.Image, x: int, y: int, width: int, height: int, color: tuple) -> None:
    """
    Draws a shape on the specified image.

    Args:
    image (Image): The image to draw on.
    x (int): The x-coordinate of the shape.
    y (int): The y-coordinate of the shape.
    width (int): The width of the shape.
    height (int): The height of the shape.
    color (tuple): The color of the shape.
    """
    draw = ImageDraw.Draw(image)
    draw.ellipse((x, y, x + width, y + height), fill=color)


def add_text(image: Image.Image, text: str, x: int, y: int) -> None:
    """
    Adds text to the specified image.

    Args:
    image (Image): The image to add text to.
    text (str): The text to add.
    x (int): The x-coordinate of the text.
    y (int): The y-coordinate of the text.
    """
    draw = ImageDraw.Draw(image)
    try:
        font = ImageFont.truetype("arial.ttf", 20)  # Use Arial font if available
    except IOError:
        font = ImageFont.load_default()  # Fallback to default font if Arial is unavailable
    draw.text((x, y), text, fill=(0, 0, 0), font=font)


def main() -> None:
    """
    Creates an image with a lion and a cat, and saves it to a file.
    """
    try:
        width, height = 800, 600
        image = create_image(width, height)
        draw_shape(image, 100, 100, 100, 100, (255, 128, 0))  # Orange color for lion
        draw_shape(image, 300, 300, 50, 50, (0, 128, 255))  # Blue color for cat
        add_text(image, 'Lion and Cat', 350, 50)
        image.show()
        image.save('lion_and_cat_fixed.png')
        print("Image created and saved as 'lion_and_cat_fixed.png'.")
    except ValueError as e:
        print(f'Invalid image size: {e}')
    except IOError as e:
        print(f'Error saving image: {e}')
    except Exception as e:
        print(f'An error occurred: {e}')


if __name__ == "__main__":
    main()

Image created and saved as 'lion_and_cat_fixed.png'.
